In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Data Import


In [2]:
def load_data(sub_sample=True, add_outlier=False):
    """Load data"""
    X = np.genfromtxt(
        "train.csv", delimiter=",", skip_header=1)[:, 2:]
    
    y = np.genfromtxt(
        "train.csv", delimiter=",", skip_header=1, usecols=[1],
        converters={1: lambda x: 0 if b"s" in x else 1})
    return X, y

In [3]:
X_train, y_train = load_data()
X_test, y_test = load_data()

OSError: train.csv not found.

In [4]:
def missing_values(X): 
      N, D = X.shape
      missing_data = np.zeros(D)
      delete = [] 
      for feature in range(D):
            missing_data[feature] = np.count_nonzero(X[:,feature]==-999)/N
            if missing_data[feature]>0.3: 
            # delete features with more than 30% missing values
                  delete.append(feature)
            elif missing_data[feature]>0:
            # complete features with less than30% missing values
                  mean = np.where(X[:,feature]==-999, 0, X[:,feature]).mean()
                  X[:,feature] = np.where(X[:,feature]==-999, mean, X[:,feature])
      X = np.delete(X, delete, 1)
      # adding an intercepta
      X = np.c_[np.ones((X.shape[0], 1)), X]

      return X

In [5]:
X_train = missing_values(X_train)
X_test = missing_values(X_test)

# Linear Classifier


In [7]:
def compute_loss(y_tr,tx_tr,weight):
    return np.linalg.norm(y_tr-tx_tr.dot(weight))/len(y_tr)

def least_squares(y, tx):
    """calculate the least squares solution."""
    w = np.linalg.solve (tx.T.dot(tx),tx.T.dot(y))
    mse = compute_loss(y, tx, w)
    return mse , w 

In [8]:
mse_linear, w_linear = least_squares(y_train,X_train)

In [9]:
pred = []
for i in range(X_test.shape[0]):
    pred.append([350000+i, X_test[i]@w_linear])

np.savetxt("firstprediction.csv", pred, fmt='%10i',delimiter=",")

# Logistic Regression (with SGD)

In [10]:
def sigmoid(t):
    return np.exp(t)/(1+np.exp(t))

def calculate_loss(y, tx, w):
  pred = sigmoid(tx.dot(w))
  loss = -y.T.dot(np.log(pred))+(1-y).T.dot(np.log(1-pred))
  return np.squeeze(-loss)

def calculate_gradient(y, tx, w):
    return tx.T.dot(sigmoid(tx.dot(w))-y)

In [11]:
def learning_by_stochastic_gradient_descent(y, tx, w, gamma,minibatch_y,minibatch_tx):
    """
    Do one step of stochastic gradient descenr using logistic regression.
    Return the loss and the updated w.
    """
    loss = compute_loss(y, tx, w)
    grad = compute_stoch_gradient(minibatch_y, minibatch_tx, w)
    w = w - gamma * grad
    return loss, w

In [12]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]



In [13]:
def logistic_regression_gradient_descent_demo(y, x, batch_size, max_iter = 10000,threshold = 1e-8,gamma = 0.01):
    # init parameters
    losses = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
          loss, w = learning_by_stochastic_gradient_descent(y, tx, w, gamma,minibatch_y,minibatch_tx)
          # log info
          if iter % 100 == 0:
              print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
          # converge criterion
          losses.append(loss)
          if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
              break
    # visualization
    visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_gradient_descent")
    print("loss={l}".format(l=calculate_loss(y, tx, w)))

In [15]:
logistic_regression_gradient_descent_demo(y_train,X_train,100)

MemoryError: Unable to allocate 466. GiB for an array with shape (250000, 250000) and data type float64